In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\End_to_End_ml_project_chest_CT_scan\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\End_to_End_ml_project_chest_CT_scan'

In [5]:
from dataclasses import dataclass, field
from pathlib import Path
from box import Box


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float
    params_callbacks: Box ## Useing Box for nested YAML data

In [6]:
from chestCancerClassifier.constants import *
from chestCancerClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
import os
from pathlib import Path

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_traning_config(self) -> TrainingConfig:
        traning = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.Unzip_dir, "chest_CT_scan_data")
        create_directories([
            Path(traning.root_dir)
        ])

        # Access the entire nested CALLBACKS section
        callbacks_params = params.CALLBACKS
        
        traning_config = TrainingConfig(
            root_dir=Path(traning.root_dir),
            trained_model_path=Path(traning.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
            
            # Pass the nested parameters to the dataclass
            params_callbacks=callbacks_params
        )

        return traning_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
tf.config.run_functions_eagerly(True)

In [9]:
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.callbacks import ReduceLROnPlateau

class Training:
    def __init__(self, config):
        self.config = config
        self.model = None

    def get_base_model(self):
        # Load the model's architecture and weights.
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    # A dedicated method to compile the model.
    def compile_model(self):
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        # ... ( code for setting up generators) ...
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split= 0.20
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        # Access the nested parameters from the config
        callback_params = self.config.params_callbacks.ReduceLROnPlateau

        # Use the parameters from your config to define the callback
        lr_scheduler = ReduceLROnPlateau(
            monitor=callback_params.monitor,
            factor=callback_params.factor,
            patience=callback_params.patience,
            min_lr=callback_params.min_lr
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
            callbacks=[lr_scheduler]
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_traning_config()
    training = Training(config=training_config)
    
    # 1. Load the model's architecture and weights.
    training.get_base_model() 
    
    # 2. Re-compile the model. This is the crucial step.
    training.compile_model() 
    
    # 3. Set up the data generators.
    training.train_valid_generator()
    
    # 4. Start the training process.
    training.train()
    
except Exception as e:
    raise e

[2025-09-22 14:00:05,275: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-22 14:00:05,293: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-22 14:00:05,297: INFO: common: created directory at: artifacts]
[2025-09-22 14:00:05,298: INFO: common: created directory at: artifacts\training]
Found 178 images belonging to 4 classes.
Found 722 images belonging to 4 classes.
Epoch 1/5


d:\Data_science\Projects\End_to_End_ml_project_chest_CT_scan\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


45/45 [==============================] - 630s 14s/step - loss: 245.7043 - accuracy: 0.2720 - val_loss: 415.0790 - val_accuracy: 0.2727 - lr: 0.1000
Epoch 2/5
45/45 [==============================] - 618s 14s/step - loss: 208.0029 - accuracy: 0.3116 - val_loss: 200.9947 - val_accuracy: 0.3864 - lr: 0.1000
Epoch 3/5
45/45 [==============================] - 618s 14s/step - loss: 208.5338 - accuracy: 0.3215 - val_loss: 160.9161 - val_accuracy: 0.3182 - lr: 0.1000
Epoch 4/5
45/45 [==============================] - 618s 14s/step - loss: 138.9649 - accuracy: 0.4108 - val_loss: 153.3569 - val_accuracy: 0.2670 - lr: 0.1000
Epoch 5/5
45/45 [==============================] - 629s 14s/step - loss: 152.5605 - accuracy: 0.3917 - val_loss: 127.9202 - val_accuracy: 0.3750 - lr: 0.1000
